In [1]:
from pathlib import Path
import os
import re
import pandas as pd
from openpyxl import load_workbook


In [2]:
# 리포지토리 루트 = notebooks의 부모
BASE_DIR = Path.cwd().resolve().parent        # .../전국도시가스보급률
DATA_DIR = BASE_DIR / "data"
OUT_DIR  = BASE_DIR / "out"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SRC     = DATA_DIR / "5. 보급률실적 (1992~2023).xlsx"
OUT_CSV = OUT_DIR  / "보급률_tidy_(2006-2023).csv"
TMP     = DATA_DIR / f"{SRC.stem}_tmp_unmerged.xlsx"  # 임시파일은 data 옆에


# 처리할 시트 인덱스(0-based) : 2~4번째 시트 => 1, 2, 3
TARGET_SHEETS = [1, 2, 3]

# ===== 1) 병합 해제 후 임시 저장(대상 시트만) =====
TMP = os.path.splitext(SRC)[0] + "_tmp_unmerged.xlsx"
wb = load_workbook(SRC)
for sidx in TARGET_SHEETS:
    ws = wb.worksheets[sidx]
    for rng in list(ws.merged_cells.ranges):
        ws.unmerge_cells(str(rng))
wb.save(TMP)

In [3]:
def tidy_from_sheet(sheet_index: int) -> pd.DataFrame:
    """병합 해제된 TMP에서 sheet_index 시트를 tidy DF로 변환"""
    raw = pd.read_excel(TMP, sheet_name=sheet_index, header=None)

    # --- 좌표(0-based): 시트 구조 동일하다고 가정 ---
    ROW_LABELS = 2     # "시도별/회사별/..." 라벨 행
    ROW_YEARS  = 3     # 연도 행
    ROW_DATA   = 4     # 데이터 시작 행

    COL_SIDO   = 0
    COL_COMP   = 1
    COL_SEDAE  = list(range(2, 8))    # C:H (세대수)
    COL_SOYO   = list(range(8, 14))   # I:N (수요가수)
    COL_RATE   = list(range(14, 20))  # O:T (보급률)

    # --- 풋노트(설명) 행 제거: 첫 열 텍스트에 패턴이 보이면 그 행부터 아래 전부 삭제 ---
    note_pat = re.compile(r"(보급률\s*산정기준|주민등록\s*세대수|공급지역\s*허가|공급권역)", re.I)
    col0 = raw.iloc[:, COL_SIDO].astype(str).fillna("")
    note_rows = col0[col0.str.contains(note_pat)].index
    if len(note_rows) > 0:
        raw = raw.iloc[: note_rows.min(), :]

    # --- 베이스(시도/회사) ---
    base = raw.iloc[ROW_DATA:, [COL_SIDO, COL_COMP]].copy()
    base.columns = ["시도", "회사"]
    base["시도"] = base["시도"].ffill()
    base["회사"] = base["회사"].ffill()

    # --- 연도 라벨 ---
    def year_labels(cols):
        years = raw.iloc[ROW_YEARS, cols].tolist()
        return [int(float(y)) for y in years]

    years_sedae = year_labels(COL_SEDAE)
    years_soyo  = year_labels(COL_SOYO)
    years_rate  = year_labels(COL_RATE)

    # --- 블록 추출 & 숫자형 변환 ---
    def block(cols, new_cols):
        dfb = raw.iloc[ROW_DATA:, cols].copy()
        dfb.columns = new_cols
        return dfb.apply(
            lambda s: pd.to_numeric(
                s.astype(str).str.replace(",", "", regex=False).str.strip(),
                errors="coerce"
            )
        )

    sedae = block(COL_SEDAE, years_sedae)
    soyo  = block(COL_SOYO,  years_soyo)
    rate  = block(COL_RATE,  years_rate)

    # --- long-format → 결합 ---
    sedae_l = pd.concat([base, sedae], axis=1).melt(
        id_vars=["시도", "회사"], var_name="연도", value_name="세대수"
    )
    soyo_l = pd.concat([base, soyo], axis=1).melt(
        id_vars=["시도", "회사"], var_name="연도", value_name="수요가수"
    )
    rate_l = pd.concat([base, rate], axis=1).melt(
        id_vars=["시도", "회사"], var_name="연도", value_name="보급률"
    )

    out = (
        sedae_l
        .merge(soyo_l, on=["시도", "회사", "연도"])
        .merge(rate_l, on=["시도", "회사", "연도"])
    )

    # --- 합계/계 제거 ---
    drop_patterns = r"(소\s*계|수도권\s*계|지\s*방\s*계|전\s*국\s*계|\(계\))"
    mask = (
        out["시도"].astype(str).str.contains(drop_patterns)
        | out["회사"].astype(str).str.contains(drop_patterns)
    )
    out = out[~mask].copy()

    # --- 시도/회사 공백 완전 제거 & 정리 ---
    out["연도"] = out["연도"].astype(int)
    out["시도"] = out["시도"].astype(str).str.replace(r"\s+", "", regex=True)
    out["회사"] = out["회사"].astype(str).str.replace(r"\s+", "", regex=True)

    return (
        out[["연도", "시도", "회사", "세대수", "수요가수", "보급률"]]
        .sort_values(["연도", "시도", "회사"])
        .reset_index(drop=True)
    )


In [4]:
# ===== 시트별 변환 → 합치기 =====
dfs = [tidy_from_sheet(sidx) for sidx in TARGET_SHEETS]
final = (
    pd.concat(dfs, ignore_index=True)
    .sort_values(["연도", "시도", "회사"])
    .reset_index(drop=True)
)

C:\Users\user\AppData\Local\Temp\ipykernel_43192\236578158.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  note_rows = col0[col0.str.contains(note_pat)].index
C:\Users\user\AppData\Local\Temp\ipykernel_43192\236578158.py:73: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  out["시도"].astype(str).str.contains(drop_patterns)
C:\Users\user\AppData\Local\Temp\ipykernel_43192\236578158.py:74: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  | out["회사"].astype(str).str.contains(drop_patterns)
C:\Users\user\AppData\Local\Temp\ipykernel_43192\236578158.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  note_rows = col0[col0.str.contains(note_pa

In [5]:
# ===== CSV 저장 =====
final.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ 완료: {OUT_CSV} (rows={len(final)})")

✅ 완료: D:\Project\전국도시가스보급률\out\보급률_tidy_(2006-2023).csv (rows=810)
